<a href="https://colab.research.google.com/github/n-nooobu/atcoder/blob/master/src/pubmedbert_base_uncased_abstract_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# インポートと準備

In [1]:
import os
import glob
import json
import math
import random
import time
import string
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import transformers as T
T.logging.set_verbosity_error()
import pytorch_lightning as pl
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
from sklearn.metrics import fbeta_score
from sklearn.model_selection import StratifiedKFold

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
def seed_torch(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

# Dataset

In [4]:
class BertDataset(Dataset):
    def __init__(self, df, model_name, include_labels=True):
        tokenizer = T.BertTokenizer.from_pretrained(model_name)

        self.df = df
        self.include_labels = include_labels

        self.sentences = df["title"].tolist()
        self.encoded = tokenizer.batch_encode_plus(
            self.sentences,
            padding = 'max_length',            
            max_length = 75,
            truncation = True,
            return_attention_mask=True
        )
        
        if self.include_labels:
            self.labels = df["judgement"].values

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        input_ids = torch.tensor(self.encoded['input_ids'][idx])
        attention_mask = torch.tensor(self.encoded['attention_mask'][idx])

        if self.include_labels:
            label = torch.tensor(self.labels[idx]).float()
            return input_ids, attention_mask, label

        return input_ids, attention_mask

# Data Module

In [5]:
class BertDataModule(pl.LightningDataModule):
    def __init__(self, model_name, data_dir, output_dir, data, fold, batch_size):
        super().__init__()

        self.model_name = model_name
        self.data_dir = data_dir
        self.output_dir = output_dir
        self.data = data
        self.fold = fold
        self.batch_size = batch_size

    def prepare_data(self):
        if os.path.isfile(self.output_dir + "train_folds.csv"):
            return

        train_df = pd.read_csv(self.data_dir + "train_" + self.data + ".csv")
        
        Fold = StratifiedKFold(n_splits=6, shuffle=True, random_state=SEED)
        for n, (train_index, val_index) in enumerate(Fold.split(train_df, train_df["judgement"])):
            train_df.loc[val_index, "fold"] = int(n)  
        train_df["fold"] = train_df["fold"].astype(np.uint8)

        if not os.path.exists(self.output_dir):
            os.makedirs(self.output_dir)
        train_df.to_csv(self.output_dir + "train_folds.csv", index=False)

    def setup(self, stage=None):
        train_df = pd.read_csv(self.output_dir + "train_folds.csv")

        trn_idx = train_df[train_df["fold"] != self.fold][train_df["fold"] != 5].index
        val_idx = train_df[train_df["fold"] == self.fold].index

        train_folds = train_df.loc[trn_idx].reset_index(drop=True)
        valid_folds = train_df.loc[val_idx].reset_index(drop=True)

        self.train_dataset = BertDataset(train_folds, self.model_name)
        self.valid_dataset = BertDataset(valid_folds, self.model_name)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=0, pin_memory=True, drop_last=True)

    def val_dataloader(self):
        return DataLoader(self.valid_dataset, batch_size=self.batch_size, shuffle=False, num_workers=0, pin_memory=True, drop_last=False)

# Lightning Model

In [6]:
class BertClassification(pl.LightningModule):
    def __init__(self, seed, data, model_name, criterion, lr, batch_size, epochs, border, fold):
        super().__init__()
        
        self.save_hyperparameters()

        self.bert = T.BertModel.from_pretrained(model_name)
        self.bert.config.output_hidden_states = True
        self.dropout = nn.Dropout(self.bert.config.hidden_dropout_prob)
        self.linear = nn.Linear(self.bert.config.hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

        self.criterion = criterion

    def forward(self, input_ids, attention_mask):
        bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states = bert_output.hidden_states
        
        h12 = hidden_states[-1][:, 0].reshape((-1, 1, 768))
        h11 = hidden_states[-2][:, 0].reshape((-1, 1, 768))
        h10 = hidden_states[-3][:, 0].reshape((-1, 1, 768))
        h9  = hidden_states[-4][:, 0].reshape((-1, 1, 768))

        all_h = torch.cat([h9, h10, h11, h12], 1)
        mean_pool = torch.mean(all_h, 1)
        pooled_output = self.dropout(mean_pool)
        
        linear_output = self.linear(pooled_output)
        
        out = self.sigmoid(linear_output).squeeze()
        return out

    def training_step(self, batch, batch_idx):
        input_ids, attention_mask, labels = batch
        y_preds = self.forward(input_ids, attention_mask)
        loss = self.criterion(y_preds, labels)
        self.log('train_loss', loss)
        return {'loss': loss, 'labels': labels.detach(), 'y_preds': y_preds.detach()}
        
    def training_epoch_end(self, outputs):
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
        labels = torch.cat([x['labels'] for x in outputs], axis=0)
        y_preds = torch.cat([x['y_preds'] for x in outputs], axis=0)
        score = fbeta_score(labels.to("cpu").numpy(), np.where(y_preds.to("cpu").numpy() < self.hparams.border, 0, 1), beta=7.0)
        self.log('train_score', score)

    def validation_step(self, batch, batch_idx):
        input_ids, attention_mask, labels = batch
        y_preds = self.forward(input_ids, attention_mask)
        loss = self.criterion(y_preds, labels)
        return {'val_loss': loss, 'val_labels': labels.detach(), 'val_y_preds': y_preds.detach()}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        labels = torch.cat([x['val_labels'] for x in outputs], axis=0)
        y_preds = torch.cat([x['val_y_preds'] for x in outputs], axis=0)
        score = fbeta_score(labels.to("cpu").numpy(), np.where(y_preds.to("cpu").numpy() < self.hparams.border, 0, 1), beta=7.0)
        self.log('val_loss', avg_loss, prog_bar=True)
        self.log('val_score', score, prog_bar=True)

    """def test_step(self, batch, batch_idx):
        input_ids, attention_mask, labels = batch
        y_preds = self.forward(input_ids, attention_mask)
        loss = self.criterion(y_preds, labels)
        return {'test_loss': loss, 'test_labels': labels.detach(), 'test_y_preds': y_preds.detach()}

    def test_epoch_end(self, outputs):
        avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
        labels = torch.cat([x['test_labels'] for x in outputs], axis=0)
        y_preds = torch.cat([x['test_y_preds'] for x in outputs], axis=0)
        score = fbeta_score(labels.to("cpu").detach().numpy(), np.where(y_preds.to("cpu").detach().numpy() < self.hparams.border, 0, 1), beta=7.0)
        self.log('test_loss', avg_loss, prog_bar=True, logger=False)
        self.log('test_score', score, prog_bar=True, logger=False)"""
    
    def predict_step(self, batch, batch_idx):
        input_ids, attention_mask = batch
        return self.forward(input_ids, attention_mask)

    def configure_optimizers(self):
        return T.AdamW(self.parameters(), lr=self.hparams.lr)

# Hyper Parameters

In [7]:
SEED = 471
DATA = "v1_cat"
MODEL_NAME = "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract"
CRITERION = nn.BCELoss()
LR = 1e-5
BATCH_SIZE = 32
EPOCHS = 4
BORDER = 0.01

# ローカルランタイム
DATA_DIR = "input/systematic-review/"
OUTPUT_DIR = "output/pubmedbert_base_uncased_abstract_10_000/"

# Fine Tuning

In [8]:
seed_torch(SEED)
results = {'best_model_path': {}, 'logs': {}}

for fold in range(5):
    checkpoint = pl.callbacks.ModelCheckpoint(
        monitor='val_loss',
        mode='min',
        save_top_k=1,
        save_weights_only=True,
        dirpath=OUTPUT_DIR
    )

    logger = pl.loggers.TensorBoardLogger(OUTPUT_DIR, name="logs", default_hp_metric=False)

    trainer = pl.Trainer(
        gpus=-1 if torch.cuda.is_available() else None, 
        max_epochs=EPOCHS,
        callbacks = [checkpoint],
        logger=logger
    )

    model = BertClassification(
        seed=SEED,
        data=DATA,
        model_name=MODEL_NAME,
        criterion=CRITERION,
        lr=LR,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        border=BORDER,
        fold=fold
    )

    dm = BertDataModule(model_name=MODEL_NAME, data_dir=DATA_DIR, output_dir=OUTPUT_DIR, data=DATA, fold=fold, batch_size=BATCH_SIZE)

    trainer.fit(model, dm)

    # チェックポイントで保存したパスをjsonに書き込む
    results['best_model_path']['fold_' + str(fold)] = checkpoint.best_model_path

    # ログをjsonに書き込む
    results['logs']['fold_' + str(fold)] = {}
    fold_dir = OUTPUT_DIR + 'logs/version_' + str(fold) +'/'
    path = glob.glob(os.path.join(fold_dir, 'events*'))[0]

    event_acc = EventAccumulator(path, size_guidance={'scalars': 0})
    event_acc.Reload()

    for tag in event_acc.Tags()['scalars']:
        events = event_acc.Scalars(tag)
        results['logs']['fold_' + str(fold)][tag] = [event.value for event in events]

# 全foldのval_scoreの平均をjsonに書き込む
val_score_list = []
for fold in range(5):
    index = results['logs']['fold_' + str(fold)]['val_loss'].index(min(results['logs']['fold_' + str(fold)]['val_loss']))
    val_score_list.append(results['logs']['fold_' + str(fold)]['val_score'][index])
results['logs']['avg_val_score'] = np.mean(val_score_list)

with open(OUTPUT_DIR + 'results.json', 'w') as f:
    json.dump(results, f, indent=4, sort_keys=True)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Missing logger folder: output/pubmedbert_base_uncased_abstract_10_000/logs

  | Name      | Type      | Params
----------------------------------------
0 | bert      | BertModel | 109 M 
1 | dropout   | Dropout   | 0     
2 | linear    | Linear    | 769   
3 | sigmoid   | Sigmoid   | 0     
4 | criterion | BCELoss   | 0     
----------------------------------------
109 M     Trainable params
0         Non-trainable params
109 M     Total params
437.932   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

# Valid Predictions

In [9]:
train_df = pd.read_csv(OUTPUT_DIR + "train_folds.csv")

with open(OUTPUT_DIR + "results.json") as f:
    results = json.load(f)

for fold in range(5):
    val_idx = train_df[train_df["fold"] == fold].index
    valid_folds = train_df.loc[val_idx].reset_index(drop=True)
    valid_dataset = BertDataset(valid_folds, MODEL_NAME, include_labels=False)
    valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)

    trained_model = BertClassification.load_from_checkpoint(checkpoint_path=results['best_model_path']['fold_' + str(fold)]).to(device)
    trained_model.eval()
    trained_model.freeze()

    y_preds = trainer.predict(model=trained_model, dataloaders=valid_loader)
    y_preds = torch.cat([x for x in y_preds]).to("cpu").detach().numpy()

    train_df.loc[train_df["fold"] == fold, "predictions"] = y_preds

train_df.to_csv(OUTPUT_DIR + "train_folds.csv", index=False)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 565it [00:00, ?it/s]

e:\nobu\programs\signate-systematic-review\venv\lib\site-packages\pytorch_lightning\core\datamodule.py:423: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 565it [00:00, ?it/s]

e:\nobu\programs\signate-systematic-review\venv\lib\site-packages\pytorch_lightning\core\datamodule.py:423: LightningDeprecationWarning: DataModule.teardown has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.teardown.
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 565it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 565it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 565it [00:00, ?it/s]

# Border Optimization

In [10]:
train_df = pd.read_csv(OUTPUT_DIR + "train_folds.csv")

borders = []
for fold in range(5):
    predictions = train_df[train_df["fold"] == fold]["predictions"].values
    labels = train_df[train_df["fold"] == fold]["judgement"].values

    score_max = 0
    for border in np.arange(0, 1, 0.001):
        score = fbeta_score(labels, np.where(predictions < border, 0, 1), beta=7.0)
        if score > score_max:
            score_max = score
            border_opt = border
            print(f"Score: {score_max:<.5f}, border: {border_opt}")
    
    borders.append(border_opt)
    print(borders)

border_avg = np.mean(borders)
print(border_avg)

results['borders'] = borders
results['0_avg_ensamble'] = {'border': border_avg}

with open(OUTPUT_DIR + 'results.json', 'w') as f:
    json.dump(results, f, indent=4, sort_keys=True)

Score: 0.54532, border: 0.0
Score: 0.77316, border: 0.001
Score: 0.82692, border: 0.002
Score: 0.86045, border: 0.003
Score: 0.87704, border: 0.004
Score: 0.87904, border: 0.005
Score: 0.88574, border: 0.006
Score: 0.89031, border: 0.007
[0.007]
Score: 0.54297, border: 0.0
Score: 0.74861, border: 0.001
Score: 0.80620, border: 0.002
Score: 0.83280, border: 0.003
Score: 0.85034, border: 0.004
Score: 0.85470, border: 0.005
Score: 0.86465, border: 0.006
Score: 0.87366, border: 0.007
Score: 0.87972, border: 0.008
Score: 0.88616, border: 0.009000000000000001
Score: 0.89056, border: 0.01
Score: 0.89578, border: 0.011
Score: 0.90168, border: 0.012
Score: 0.90435, border: 0.013000000000000001
Score: 0.90719, border: 0.014
Score: 0.90941, border: 0.015
Score: 0.91164, border: 0.016
Score: 0.91324, border: 0.017
Score: 0.91356, border: 0.018000000000000002
Score: 0.91582, border: 0.019
Score: 0.91792, border: 0.02
Score: 0.91889, border: 0.021
Score: 0.92019, border: 0.022
Score: 0.92182, border:

In [11]:
val_scores = []
for fold in range(5):
    predictions = train_df[train_df["fold"] == fold]["predictions"].values
    labels = train_df[train_df["fold"] == fold]["judgement"].values

    val_scores.append(fbeta_score(labels, np.where(predictions < border_avg, 0, 1), beta=7.0))

print(val_scores)
print(np.mean(val_scores))

results['0_avg_ensamble']['val_score'] = val_scores
results['0_avg_ensamble']['avg_val_score'] = np.mean(val_scores)

with open(OUTPUT_DIR + 'results.json', 'w') as f:
    json.dump(results, f, indent=4, sort_keys=True)

[0.8740313570012616, 0.8899537908608591, 0.85431654676259, 0.8469895046952679, 0.8171695402298852]
0.8564921479099727


# Fold 5 Test

In [12]:
train_df = pd.read_csv(OUTPUT_DIR + "train_folds.csv")
f5_idx = train_df[train_df["fold"] == 5].index
f5_df = train_df.loc[f5_idx].reset_index(drop=True)
f5_dataset = BertDataset(f5_df, MODEL_NAME, include_labels=False)
f5_loader = DataLoader(f5_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)

for fold in range(5):
    trained_model = BertClassification.load_from_checkpoint(checkpoint_path=results['best_model_path']['fold_' + str(fold)]).to(device)
    trained_model.eval()
    trained_model.freeze()

    y_preds = trainer.predict(model=trained_model, dataloaders=f5_loader)
    y_preds = torch.cat([x for x in y_preds]).to("cpu").detach().numpy()

    train_df.loc[train_df["fold"] == 5, 'fold_' + str(fold)] = y_preds

train_df.to_csv(OUTPUT_DIR + "train_folds.csv", index=False)

e:\nobu\programs\signate-systematic-review\venv\lib\site-packages\pytorch_lightning\core\datamodule.py:423: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 565it [00:00, ?it/s]

e:\nobu\programs\signate-systematic-review\venv\lib\site-packages\pytorch_lightning\core\datamodule.py:423: LightningDeprecationWarning: DataModule.teardown has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.teardown.
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 565it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 565it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 565it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 565it [00:00, ?it/s]

## Average Ensamble Score

In [13]:
y_predictions = []
for fold in range(5):
    y_preds = train_df.loc[train_df["fold"] == 5, 'fold_' + str(fold)].values
    y_predictions.append(y_preds)

y_predictions = np.mean(y_predictions, axis=0)

predictions = np.where(y_predictions < border_avg, 0, 1)
labels = train_df.loc[train_df["fold"] == 5, "judgement"].values
f5_score = fbeta_score(labels, predictions, beta=7.0)

print(f5_score)

results['0_avg_ensamble']['f5_score'] = f5_score

with open(OUTPUT_DIR + 'results.json', 'w') as f:
    json.dump(results, f, indent=4, sort_keys=True)

train_df.loc[train_df["fold"] == 5, 'avg_ensamble_y_predictions'] = y_predictions
train_df.loc[train_df["fold"] == 5, 'avg_ensamble_predictions'] = predictions
train_df.to_csv(OUTPUT_DIR + "train_folds.csv", index=False)

0.8876779750395499


## Vote Ensamble Score

In [14]:
borders = results['borders']

predictions = []
for fold in range(5):
    preds = np.where(train_df.loc[train_df["fold"] == 5, 'fold_' + str(fold)].values < borders[fold], 0, 1)
    train_df.loc[train_df["fold"] == 5, 'fold_' + str(fold) + '_predictions'] = preds
    predictions.append(preds)

predictions = np.sum(predictions, axis=0)
train_df.loc[train_df["fold"] == 5, 'vote_ensamble_votes'] = predictions

predictions = np.where(predictions < 2.5, 0, 1)
train_df.loc[train_df["fold"] == 5, 'vote_ensamble_predictions'] = predictions
labels = train_df.loc[train_df["fold"] == 5, "judgement"].values
f5_score = fbeta_score(labels, predictions, beta=7.0)

print(f5_score)

results['1_vote_ensamble'] = {'f5_score': f5_score}

with open(OUTPUT_DIR + 'results.json', 'w') as f:
    json.dump(results, f, indent=4, sort_keys=True)

train_df.to_csv(OUTPUT_DIR + "train_folds.csv", index=False)

0.8700284090909093


In [15]:
train_df[train_df["fold"] == 5].to_csv(OUTPUT_DIR + "f5_predictions.csv", index=False)

# Inference

In [16]:
test = pd.read_csv(DATA_DIR + "test_" + DATA + ".csv")
test_dataset = BertDataset(test, MODEL_NAME, include_labels=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)

sub = pd.read_csv(DATA_DIR + "sample_submit.csv", header=None)
sub.columns = ["id", "judgement"]

with open(OUTPUT_DIR + "results.json") as f:
    results = json.load(f)

for fold in range(5):
    trained_model = BertClassification.load_from_checkpoint(checkpoint_path=results['best_model_path']['fold_' + str(fold)]).to(device)
    trained_model.eval()
    trained_model.freeze()

    y_preds = trainer.predict(model=trained_model, dataloaders=test_loader)
    y_preds = torch.cat([x for x in y_preds]).to("cpu").detach().numpy()

    sub['fold_' + str(fold)] = y_preds

sub.to_csv(OUTPUT_DIR + "predictions.csv", index=False)

e:\nobu\programs\signate-systematic-review\venv\lib\site-packages\pytorch_lightning\core\datamodule.py:423: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 565it [00:00, ?it/s]

e:\nobu\programs\signate-systematic-review\venv\lib\site-packages\pytorch_lightning\core\datamodule.py:423: LightningDeprecationWarning: DataModule.teardown has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.teardown.
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 565it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 565it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 565it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 565it [00:00, ?it/s]

## Average Ensamble Submission

In [17]:
sub = pd.read_csv(DATA_DIR + "sample_submit.csv", header=None)
sub.columns = ["id", "judgement"]

predictions = pd.read_csv(OUTPUT_DIR + "predictions.csv")

preds = []
for fold in range(5):
    preds.append(predictions['fold_' + str(fold)].values)

preds = np.mean(preds, axis=0)

sub["judgement"] = preds
sub.to_csv(OUTPUT_DIR + "predictions_avg.csv", index=False, header=False)

border = results['0_avg_ensamble']['border']
sub["judgement"] = np.where(preds < border, 0, 1)
sub.to_csv(OUTPUT_DIR + "submission_avg_" + str(border)[2: 6] + ".csv", index=False, header=False)

## Vote Ensamble Submission

In [18]:
sub = pd.read_csv(DATA_DIR + "sample_submit.csv", header=None)
sub.columns = ["id", "judgement"]

predictions = pd.read_csv(OUTPUT_DIR + "predictions.csv")

with open(OUTPUT_DIR + "results.json") as f:
    results = json.load(f)

borders = results['borders']

preds = []
for fold in range(5):
    preds.append(np.where(predictions['fold_' + str(fold)].values < borders[fold], 0, 1))

preds = np.sum(preds, axis=0)

sub["judgement"] = preds
sub.to_csv(OUTPUT_DIR + "predictions_vote.csv", index=False, header=False)

preds = np.where(preds < 2.5, 0, 1)

sub["judgement"] = preds
sub.to_csv(OUTPUT_DIR + "submission_vote.csv", index=False, header=False)

# Tensorboard

In [19]:
#!taskkill /IM "tensorboard.exe" /F

In [20]:
#%load_ext tensorboard
#%tensorboard --logdir ./